# Data preparation

In [1]:
import vaex
df_taxi_trips_all = vaex.open('./chunks/taxi_*.hdf5')

### Replace spaces and uppercases in column names
column_names = df_taxi_trips_all.column_names
column_names_refactored = [ln.replace(' ', '_').lower() for ln in column_names]

for i, column in enumerate(column_names):
    df_taxi_trips_all.rename(column, column_names_refactored[i])
    
df_taxi_trips_all.head()

#,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid__location
0,7ce98ec3d6d2b81bf6a91a5333e84690ec4e94f3,'30ac065db3b331d97b0842f0cb1740ed5b8d7fe26530441...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,480,1.5,nan,1.70311e+10,nan,8,7.5,0,0,1.5,9,Cash,Dispatch Taxi Affiliation,nan,nan,--,41.9059,-87.6309,POINT (-87.6308650266 41.9058577688)
1,e447e5c2e41c4f6751907d5e686f35b34e315afb,'4e902eb187e3942c1da821c8aa349fc848c6ff3f71f1070...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,0,0,nan,nan,nan,nan,27,0,0,0,27,Credit Card,Suburban Dispatch LLC,nan,nan,--,nan,nan,--
2,95e39db90a6a468a863f43c8eccce7e26ba3db39,'78a7a28fe001257044d5095fcb1ff7156a47411ff93cc5e...,2017-01-01 00:00:00.000000000,2017-01-01 00:45:00.000000000,2280,0.2,nan,nan,nan,76,22.25,0,0,0,22.25,Cash,Dispatch Taxi Affiliation,nan,nan,--,41.9803,-87.9136,POINT (-87.913624596 41.9802643146)
3,b09914181a9bd6a5dcd2e44941d215bc663763df,'c157870bbd395a664de1f3e6f7868b3b89472295c4e2d6e...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,480,1.7,nan,nan,nan,nan,7.5,0,0,0,7.5,Cash,4787 - 56058 Reny Cab Co,nan,nan,--,nan,nan,--
4,d039b0e5600d5008f95bd55114fe8acdeba81d99,'b5c4dfe366c83cef5604db1afcfbc15c8c38eeefd896b98...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,540,0,nan,nan,nan,nan,6.75,0,0,1,7.75,Cash,Blue Ribbon Taxi Association Inc.,nan,nan,--,nan,nan,--
5,94291f34710a6ee268b32bd3da208f6cbb65a0de,'ff6aa87ad2487541184808bdb9202867f5ec8ff5db6adf5...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,0,0,nan,nan,nan,nan,6.25,0,0,0,6.25,Credit Card,Suburban Dispatch LLC,nan,nan,--,nan,nan,--
6,f00546971a3e4c8ab55761a337443a0dfa33f42f,'631de3a4ad35b7d7d75949b33ad59e09820b51b3f58225d...,2017-01-01 00:00:00.000000000,2017-01-01 00:30:00.000000000,1800,0,1.70311e+10,1.70311e+10,8,8,13.25,4.25,0,1,18.5,Credit Card,Taxi Affiliation Services,41.8932,-87.6378,POINT (-87.6378442095 41.8932163595),41.8925,-87.6262,POINT (-87.6262149064 41.8925077809)
7,fe9bfc25af9dffac30370f4e7d57a8c8a8bdbc0d,'b7da4d1247a408c100b4964f236760694290aace75c8b87...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,660,2.7,nan,nan,2,3,10,2,0,0,12,Credit Card,Choice Taxi Association,42.0016,-87.695,POINT (-87.6950125892 42.001571027),41.9658,-87.6559,POINT (-87.6558787862 41.96581197)
8,42b662e1238bb7dceccca2b19d395df44dc12dd5,'5985b42dd0cbd4492cd51df71c1fd5dbed60edb96d589b7...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,0,0,nan,nan,nan,nan,7.5,3,0,0,10.5,Credit Card,Suburban Dispatch LLC,nan,nan,--,nan,nan,--
9,89b4ff599f5d23fa177b8bd915b7aa3c2e0608b3,'29051a8f04056bb47ab86564c9fec32eb9e15e8ce6aed24...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,0,0,nan,nan,nan,nan,21.75,3.25,0,0,25,Credit Card,T.A.S. - Payment Only,nan,nan,--,nan,nan,--


In [2]:
# open external data
df_census_tracts = vaex.open('./data/chicago_census_tracts.csv')
df_census_tracts.head()

#,the_geom,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,COMMAREA,COMMAREA_N,NOTES
0,'MULTIPOLYGON (((-87.62404799998049 41.730216999...,17,31,842400,17031842400,8424,Census Tract 8424,44,44,
1,'MULTIPOLYGON (((-87.6860799999848 41.8229560000...,17,31,840300,17031840300,8403,Census Tract 8403,59,59,
2,'MULTIPOLYGON (((-87.62934700001182 41.852797000...,17,31,841100,17031841100,8411,Census Tract 8411,34,34,
3,'MULTIPOLYGON (((-87.68813499997718 41.855690999...,17,31,841200,17031841200,8412,Census Tract 8412,31,31,
4,'MULTIPOLYGON (((-87.63312200003458 41.874488000...,17,31,839000,17031839000,8390,Census Tract 8390,32,32,
5,'MULTIPOLYGON (((-87.6678199999753 41.8741839999...,17,31,838200,17031838200,8382,Census Tract 8382,28,28,
6,'MULTIPOLYGON (((-87.73706400002477 41.771203999...,17,31,650301,17031650301,6503.01,Census Tract 6503.01,65,65,
7,'MULTIPOLYGON (((-87.64386399998179 41.663210000...,17,31,530503,17031530503,5305.03,Census Tract 5305.03,53,53,
8,'MULTIPOLYGON (((-87.83844200004106 41.970199999...,17,31,760803,17031760803,7608.03,Census Tract 7608.03,76,76,
9,'MULTIPOLYGON (((-87.6543830000042 41.9902020000...,17,31,30601,17031030601,306.01,Census Tract 306.01,77,77,


In [3]:
df_community_areas = vaex.open('./data/community_areas.csv')
df_community_areas.head()

#,the_geom,PERIMETER,AREA,COMAREA_,COMAREA_ID,AREA_NUMBE,COMMUNITY,AREA_NUM_1,SHAPE_AREA,SHAPE_LEN
0,'MULTIPOLYGON (((-87.60914087617894 41.844692502...,0,0,0,0,35,DOUGLAS,35,4.60046e+07,31027.1
1,'MULTIPOLYGON (((-87.59215283879394 41.816929346...,0,0,0,0,36,OAKLAND,36,1.6914e+07,19565.5
2,'MULTIPOLYGON (((-87.62879823733725 41.801893033...,0,0,0,0,37,FULLER PARK,37,1.99167e+07,25339.1
3,'MULTIPOLYGON (((-87.6067081256125 41.8168137705...,0,0,0,0,38,GRAND BOULEVARD,38,4.84925e+07,28196.8
4,'MULTIPOLYGON (((-87.59215283879394 41.816929346...,0,0,0,0,39,KENWOOD,39,2.90717e+07,23325.2
5,'MULTIPOLYGON (((-87.6744075678037 41.9761034044...,0,0,0,0,4,LINCOLN SQUARE,4,7.13523e+07,36624.6
6,'MULTIPOLYGON (((-87.60603749217005 41.785874064...,0,0,0,0,40,WASHINGTON PARK,40,4.23739e+07,28175.3
7,'MULTIPOLYGON (((-87.58037662085418 41.802525022...,0,0,0,0,41,HYDE PARK,41,4.51054e+07,29746.7
8,'MULTIPOLYGON (((-87.57714456891335 41.786146410...,0,0,0,0,42,WOODLAWN,42,5.78152e+07,46937
9,'MULTIPOLYGON (((-87.65455590025104 41.998166149...,0,0,0,0,1,ROGERS PARK,1,5.12599e+07,34052.4


### Reduce amount of columns and drop rows with null values in important columns

In [5]:
# Display amount of rows before preparation
print("Total number of rows: " + str(df_taxi_trips_all.count()))

Total number of rows: 24988003


In [6]:
# drop 'dropoff_centroid__location' and 'pickup_centroid_location' because we work with latitude and longitude values
df_taxi_trips_all_dropped = df_taxi_trips_all.drop(['dropoff_centroid__location', 'pickup_centroid_location'])

In [7]:
# drop rows with trip_seconds and trip_miles = 0 seconds and drop several columns with NaN (Not a Number) and NA (Not available) values
# We do not drop rows with null values in census tracts and community areas to cover trips which start/end outside of the marked area
df_taxi_trips_cleaned_sec = df_taxi_trips_all_dropped[df_taxi_trips_all_dropped['trip_seconds'] != 0.0]
print("Number of dropped rows with 'trip_seconds' = 0: " + str(df_taxi_trips_all.count() - df_taxi_trips_cleaned_sec.count()))

df_taxi_trips_cleaned_sec_trips = df_taxi_trips_cleaned_sec[df_taxi_trips_cleaned_sec['trip_miles'] != 0.0]
print("Number of dropped rows with 'trip_miles' = 0: " + str(df_taxi_trips_cleaned_sec.count() - df_taxi_trips_cleaned_sec_trips.count()))

df_taxi_trips_cleaned_sec_trips_location = df_taxi_trips_cleaned_sec_trips.dropnan(column_names=["pickup_centroid_latitude", "pickup_centroid_longitude", "dropoff_centroid_latitude", "dropoff_centroid_longitude"], how='any')
print("Number of dropped rows with NaN-values in 'pickup_centroid_latitude', 'pickup_centroid_longitude', 'dropoff_centroid_latitude', 'dropoff_centroid_longitude': " + str(df_taxi_trips_cleaned_sec_trips.count() - df_taxi_trips_cleaned_sec_trips_location.count()))

df_taxi_trips_prepared = df_taxi_trips_cleaned_sec_trips_location.dropna(column_names=["taxi_id", "trip_seconds", "trip_miles"], how='any')
print("Number of dropped rows with NA-values in 'taxi_id', 'trip_seconds', 'trip_miles': " + str(df_taxi_trips_cleaned_sec_trips_location.count() - df_taxi_trips_prepared.count()))

print("Number of rows after data preparation: " + str(df_taxi_trips_prepared.count()))

Number of dropped rows with 'trip_seconds' = 0: 517580
Number of dropped rows with 'trip_miles' = 0: 2531945
Number of dropped rows with NaN-values in 'pickup_centroid_latitude', 'pickup_centroid_longitude', 'dropoff_centroid_latitude', 'dropoff_centroid_longitude': 2362682
Number of dropped rows with NA-values in 'taxi_id', 'trip_seconds', 'trip_miles': 2797
Number of rows after data preparation: 19572999


### Check for consistency

In [8]:
# check if trip ids are uniqe
print("Trip IDs are unique?: " + str(len(df_taxi_trips_prepared) == df_taxi_trips_prepared['trip_id'].nunique()))

KeyboardInterrupt: 

In [ ]:
# check for consistency in community areas
print("Number of community areas: " + str(df_community_areas.count()))
print("Number of community areas in taxi trip data without NaN-values: " + str(df_taxi_trips_all.pickup_community_area.nunique(dropnan=True)))

Number of community areas: 77
Number of community areas in taxi trip data without NaN-values: 77


In [ ]:
import pyarrow as pa
community_areas = df_community_areas.AREA_NUMBE.values.unique()
community_areas_int = set([area.as_py() for area in community_areas])

community_areas_taxi_trips = df_taxi_trips_all.pickup_community_area.unique(dropnan=True)
community_areas_taxi_trips_int = set([int(area) for area in community_areas_taxi_trips])

print("Do the community area IDs in the taxi trip data match the community area dataset? ",community_areas_int == community_areas_taxi_trips_int)

Do the community area IDs in the taxi trip data match the community area dataset?  True


In [ ]:
# check for consistency in census tracts
print("Number of census tracts: " + str(df_census_tracts.count()))
print("Number of pickup census tracts in taxi trip data without NaN-values: " + str(df_taxi_trips_all.pickup_census_tract.nunique(dropnan=True)))
print("Number of dropoff census tracts in taxi trip data without NaN-values: " + str(df_taxi_trips_all.dropoff_census_tract.nunique(dropnan=True)))

# The results show that the taxi trips contain more census tracts than the dataset from the Chicago Data Portal

Number of census tracts: 801
Number of pickup census tracts in taxi trip data without NaN-values: 952
Number of dropoff census tracts in taxi trip data without NaN-values: 1074


In [ ]:
# check how many of the taxi trips census tracts ids are represented in the official data set
df_census_tracts.GEOID10.values
census_tracts = set([id.as_py() for id in df_census_tracts.GEOID10.values])
census_tracts_taxi_pickups = set([int(id) for id in df_taxi_trips_all.pickup_census_tract.unique(dropnan=True)])
census_tracts_taxi_dropoffs = set([int(id) for id in df_taxi_trips_all.dropoff_census_tract.unique(dropnan=True)])

known_pickup_ids = census_tracts.intersection(census_tracts_taxi_pickups)
known_dropoff_ids = census_tracts.intersection(census_tracts_taxi_dropoffs)

unknown_pickup_ids = census_tracts_taxi_pickups - census_tracts
unknown_dropoffs_ids = census_tracts_taxi_dropoffs - census_tracts

print("Number of known pickup ids: " + str(len(known_pickup_ids)))
print("Number of unknown pickup ids: " + str(len(unknown_pickup_ids)))
print("Number of known dropoff ids: " + str(len(known_dropoff_ids)))
print("Number of unknown dropoff ids: " + str(len(unknown_dropoffs_ids)))

Number of known pickup ids: 568
Number of unknown pickup ids: 384
Number of known dropoff ids: 656
Number of unknown dropoff ids: 418


### Hourly discretization of taxi trips

In [9]:
# create columns for hourly and 4-hourly temporal discretization
df_taxi_trips_prepared["trip_start_hour"] = df_taxi_trips_prepared["trip_start_timestamp"].dt.hour
df_taxi_trips_prepared["trip_end_hour"] = df_taxi_trips_prepared["trip_end_timestamp"].dt.hour
df_taxi_trips_prepared["trip_start_4h_period"] = df_taxi_trips_prepared["trip_start_timestamp"].dt.hour//4
df_taxi_trips_prepared["trip_end_4h_period"] = df_taxi_trips_prepared["trip_end_timestamp"].dt.hour//4

In [10]:
df_taxi_trips_prepared.head()

#,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,trip_start_hour,trip_end_hour,trip_start_4h_period,trip_end_4h_period
0,fe9bfc25af9dffac30370f4e7d57a8c8a8bdbc0d,'b7da4d1247a408c100b4964f236760694290aace75c8b87...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,660,2.7,nan,nan,2,3,10,2,0,0,12,Credit Card,Choice Taxi Association,42.0016,-87.695,41.9658,-87.6559,0,0,0,0
1,149f18c2d8b9bce6001cc9e156092aa562c83bc1,'b7a562fe2d37014bb47274cb5baa361b97ff9263e3ba526...,2017-01-01 00:00:00.000000000,2017-01-01 00:15:00.000000000,120,0.5,nan,nan,16,16,4.5,0,0,0,4.5,Cash,Taxi Affiliation Services,41.9536,-87.7235,41.9536,-87.7235,0,0,0,0
2,1b545d0be27b195076e5d2a7d4fc0ebef6ef20ac,'56a1119c6ca57e39525cf06829f9ecff553cf4b5ac24821...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,240,1.3,1.70318e+10,1.70318e+10,32,28,6.25,0,0,0,6.25,Cash,Taxi Affiliation Services,41.881,-87.6327,41.8853,-87.6572,0,0,0,0
3,c3ee40051c5643c54064a8b9df3686d2d411518d,'7987cfe5e3b2c9869665f70fe35dbe7423e7fc5852f2d6f...,2017-01-01 00:00:00.000000000,2017-01-01 00:15:00.000000000,1500,0.4,1.70318e+10,1.70311e+10,32,8,22.75,4.85,0,1.5,29.1,Credit Card,Blue Ribbon Taxi Association Inc.,41.881,-87.6327,41.8983,-87.6208,0,0,0,0
4,29f30c8a6bbc028f5d63aa18c708d31b47267a9f,'0b269137bd94b55212311226d9e1554423e12bcc16a432c...,2017-01-01 00:00:00.000000000,2017-01-01 00:30:00.000000000,1620,0.9,1.70318e+10,1.70311e+10,32,8,12.25,2.85,0,2,17.1,Credit Card,Taxi Affiliation Services,41.881,-87.6327,41.8925,-87.6262,0,0,0,0
5,bfa259279b4557566d2d06381156715d9be355a1,'7e7dce4bc47f1de9dd9e4fe427408dbf9fc61d786574a5a...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,480,3.5,nan,nan,77,6,11.25,0,0,1,12.25,Cash,Choice Taxi Association,41.9867,-87.6634,41.9442,-87.656,0,0,0,0
6,55ae4fa955e6bc8705b995248c34f4183d848c27,'d1d781b6807176ed53fbb9d0010dd0a87e29c073984bda6...,2017-01-01 00:00:00.000000000,2017-01-01 00:00:00.000000000,540,1,1.70311e+10,1.70311e+10,8,8,6.75,0,0,2,8.75,Cash,Dispatch Taxi Affiliation,41.9028,-87.6261,41.8909,-87.6189,0,0,0,0
7,ce908cd441e28d08dfb22bf168102beabb5e16e4,'f7630edc68a131c4a97382dd93206949da80ec1443d8da7...,2017-01-01 00:00:00.000000000,2017-01-01 00:15:00.000000000,480,0.5,1.70311e+10,1.70311e+10,8,8,6.25,2,0,1.5,9.75,Credit Card,Taxi Affiliation Services,41.8925,-87.6262,41.8909,-87.6189,0,0,0,0
8,a06320ba7f7246711505daf4bc236fcafd0142f2,'9680ca84d7fa9fceccc37db042e708c1fd78318d410201f...,2017-01-01 00:00:00.000000000,2017-01-01 00:15:00.000000000,660,3.3,nan,nan,29,28,11.25,0,0,0,11.25,Cash,Taxi Affiliation Services,41.8602,-87.7172,41.874,-87.6635,0,0,0,0
9,0642889773d6d10b72d613584fceba18232f0dfd,'38c926561cf2221a4785159e39aef376ef08ddb3c6081c4...,2017-01-01 00:00:00.000000000,2017-01-01 00:15:00.000000000,1140,0.3,nan,nan,44,38,17.5,0,0,0,17.5,Cash,Taxi Affiliation Services,41.7402,-87.616,41.8129,-87.6179,0,0,0,0


In [11]:
# export prepared dataframe
df_taxi_trips_prepared.export_hdf5('./chunks/prepared/trips_prepared.hdf5', chunk_size=1_000_000, progress=True)

export(hdf5) [----------------------------------------]  0.00% estimated time: unknown                 

KeyboardInterrupt: 